In [ ]:
import requests
import json


project_id = "76ae4a83-28b3-48dc-837c-f1c6fa25dd9a"

url = "https://starknet-mainnet.blastapi.io/" + project_id

In [9]:
def feed_request(url, headers, data):
    response = requests.post(url, headers=headers, data=json.dumps(data))        # Send the POST request

    if response.status_code == 200: # If the request was successful
        result = response.json()
        
    else: # If the request was unsuccessful
        print(f"Error: {response.status_code}, {response.text}")
        result = None
    
    return result

In [ ]:

# Define the headers
headers = {
    'Content-Type': 'application/json'
}

# Define the payload (same as the cURL -d parameter)
data = {
    "jsonrpc": "2.0",
    "id": 0,
    "method": "starknet_blockNumber"
}

result = feed_request(url, headers, data)

print(result.get("result"))


916128


### What is different with BlastAPI : 

It has access to most of the core API calls on Starknet

It offers a developper API which is more user-friendly than the Starknet API

In [2]:
import requests

# Define the API endpoint
url = 'https://starknet-mainnet.blastapi.io/76ae4a83-28b3-48dc-837c-f1c6fa25dd9a/builder/getWalletTokenHistory'

# Define the parameters (same as in the cURL request)
params = {
    'walletAddress': '0x03835178417676fa1bdb041ba8bae1510bda1d7515f4abd533e1d14ad74a06af'  # Replace with the actual wallet address
}

# Send the GET request with parameters
response = requests.get(url, params=params)

# Check if the response is successful
if response.status_code == 200:
    # Parse and print the JSON response
    result = response.json()
    print("Wallet Token History:", result)
else:
    print(f"Error: {response.status_code}, {response.text}")



Wallet Token History: {'walletAddress': '0x03835178417676fa1bdb041ba8bae1510bda1d7515f4abd533e1d14ad74a06af', 'count': 5, 'historicalBalances': [{'contractAddress': '0x049d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7', 'contractDecimals': '18', 'contractName': 'Ether', 'contractSymbol': 'ETH', 'balance': '4258695265060674', 'firstTransfer': {'fromAddress': '0x04c662633789a77b4ace49e0e167fccb23e87fec2ca1b4649d8aec71298dcda0', 'toAddress': '0x03835178417676fa1bdb041ba8bae1510bda1d7515f4abd533e1d14ad74a06af', 'value': '5000000000000000', 'blockHash': '0x04e2727ca5e601740b0534c067951e086f97551df8f410722ffc25ae1eb30822', 'blockNumber': 151061, 'blockTimestamp': '2023-08-12T05:28:48.000Z', 'transactionHash': '0x01984621c3d2de617c52cce289e2793e0bc7717798e51caf84444ac2215c196e'}, 'lastTransfer': {'fromAddress': '0x03835178417676fa1bdb041ba8bae1510bda1d7515f4abd533e1d14ad74a06af', 'toAddress': '0x01176a1bd84444c89232ec27754698e5d2e7e1a7f1539f12027f28b23ec9f3d8', 'value': '334211

In [3]:
balance = result['historicalBalances']


In [4]:
len(balance)


5

In [5]:
balance[1]

{'contractAddress': '0x068f5c6a61780768455de69077e07e89787839bf8166decfbf92b645209c0fb8',
 'contractDecimals': '6',
 'contractName': 'Tether USD',
 'contractSymbol': 'USDT',
 'balance': '14878243',
 'firstTransfer': {'fromAddress': '0x04270219d365d6b017231b52e92b3fb5d7c8378b05e9abc97724537a80e93b0f',
  'toAddress': '0x03835178417676fa1bdb041ba8bae1510bda1d7515f4abd533e1d14ad74a06af',
  'value': '1020185',
  'blockHash': '0x01e809def2b31a3c7545889efe6fd0a2b09c187b23b93033d341703b61116e5a',
  'blockNumber': 377241,
  'blockTimestamp': '2023-11-04T23:39:55.000Z',
  'transactionHash': '0x075896c642e6e8667685d6ea8936ac0715d8baa08712164b1c2fe180c6143578'},
 'lastTransfer': {'fromAddress': '0x04270219d365d6b017231b52e92b3fb5d7c8378b05e9abc97724537a80e93b0f',
  'toAddress': '0x03835178417676fa1bdb041ba8bae1510bda1d7515f4abd533e1d14ad74a06af',
  'value': '13858058',
  'blockHash': '0x007f6134153fbe968893acc97317d5b105db18adc98fc661419d971f9671dd34',
  'blockNumber': 642706,
  'blockTimestamp': 

In [ ]:
# Try to extract meaningful information from all the transactions in the wallet

# Define the API endpoint
url = 'https://starknet-mainnet.blastapi.io/76ae4a83-28b3-48dc-837c-f1c6fa25dd9a/builder/getWalletTokenHistory'

# Define the parameters (same as in the cURL request)
params = {
    'walletAddress': '0x03835178417676fa1bdb041ba8bae1510bda1d7515f4abd533e1d14ad74a06af'  # Replace with the actual wallet address
}

# Send the GET request with parameters
response = requests.get(url, params=params)

# Check if the response is successful
if response.status_code == 200:
    # Parse and print the JSON response
    result = response.json()
    print("Wallet Token History:", result)
else:
    print(f"Error: {response.status_code}, {response.text}")